In [4]:
# -*- coding: utf-8 -*-
"""
LOOCV (leave-one-trial-out) para o módulo epidemiológico (DSSAT/CROPGRO).

Como funciona (resumo):
- Para cada curva i (left-out):
    1) Treinamos SOMENTE os parâmetros biológicos nos P-1 ensaios restantes.
       Em cada simulação de treino, a curva usa seu NDS e DAE_START (do control).
    2) Testamos na curva i (deixada de fora) usando os biológicos treinados + NDS/DAE_START da própria curva i.
    3) Guardamos métricas (RMSE_i, EF_i, Willmott d/d1, MSE->bias²,SDSD,LCS, n_i).
- No final:
    - RMSEp = sqrt(ΣSSE_i / Σn_i)
    - EF_mean = média(EF_i)
    - Decomposição global do MSE (bias², SDSD, LCS) com todos os pontos concatenados.
    - Willmott global (d, d1) com todos os pontos concatenados.

Observação: o arquivo disease_parameters.txt é reescrito ANTES de cada execução
com (biológicos compartilhados + NDS/DAE_START da curva a ser rodada).
"""

import os
import math
import random
import subprocess
from math import ceil
from pathlib import Path
from typing import Dict, Tuple, List

import numpy as np
import pandas as pd

# -------------------- CONFIG --------------------
REGION_NAME = "sul"

CONTROL_SHEET_PATH = rf"H:\Meu Drive\mestrado_pesquisa\tables\control_{REGION_NAME}.xlsx"
OBSERVED_DATA_PATH = r"H:\Meu Drive\mestrado_pesquisa\dados\CURVE_DATABASE_Severity&Defol.xlsx"
OBSERVED_SHEET_NAME = "organizados"

PARAMS_FILE = r"C:\DSSAT48\Soybean\disease_parameters.txt"
DSSAT_EXE   = r"C:\dssat-csm-os-develop\build\bin\Debug\dscsm048.exe"
EXP_DIR     = r"C:\DSSAT48\Soybean"
DSSAT_OUTPUT_FILE = os.path.join(EXP_DIR, "DISEASE_DEVELOPMENT.OUT")

OUTPUT_DIR = Path(r"C:\Users\User\Desktop\parametros\cross_validation")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Reprodutibilidade
random.seed(42)
np.random.seed(42)

# -------------------- Sensibilidade (μ*) e FAIXAS: apenas parâmetros BIOLÓGICOS --------------------
MU_STAR = {
    'LDMin':        16.830110,
    'KVERHULST':    11.589222,
    'RVERHULST':    7.176244,
    'LESIONAGEOPT': 6.698133,
    'LESLIFEMAX':   5.972566,
    'LESION_S':     4.914001,
    'COF_A':        3.704213,
    'YMAX':         3.566737,
    'TOT_D':        2.789234,
    'TMAX_D':       1.894113,
    'COF_B':        1.716922,
    'TOT_G':        1.335044,
    'TMAX_G':       0.913451,
    'TMIN_D':       0.300000,
    'TMIN_G':       0.183453,
}

def get_bio_param_ranges():
    return {
        'LESION_S':     (1.75e-6, 2.50e-6),
        'KVERHULST':    (400000, 571400),  
        'RVERHULST':    (0.5, 4.0),
        'YMAX':         (0.20, 0.40),
        'COF_A':        (0.10, 0.30),
        'COF_B':        (1.00, 3.00),
        'TMIN_G':       (8.0, 11.0),
        'TOT_G':        (20.0, 25.0),
        'TMAX_G':       (28.5, 33.0),
        'TMIN_D':       (8.0, 12.0),
        'TOT_D':        (21.0, 25.0),
        'TMAX_D':       (30.0, 34.0),
        'LDMin':        (5.0, 16.0),
        'LESIONAGEOPT': (0.1, 0.9),
        'LESLIFEMAX':   (20.0, 70.0),
    }

RANGES_BIO = get_bio_param_ranges()

def weights_from_mu_star(mu):
    vals = np.array(list(mu.values()), dtype=float)
    if np.allclose(vals.std(), 0.0):
        return {k: 1.0 for k in mu}
    vmin, vmax = float(vals.min()), float(vals.max())
    return {k: 0.15 + 0.85 * ((mu[k] - vmin) / (vmax - vmin) if vmax > vmin else 1.0) for k in mu}

WEIGHTS = weights_from_mu_star(MU_STAR)

def nominal_from_ranges(ranges):
    nom = {k: (a + b) / 2.0 for k, (a, b) in ranges.items()}
    nom['KVERHULST'] = int(round(nom['KVERHULST']))
    return nom

NOMINAL_BIO = nominal_from_ranges(RANGES_BIO)

# Otimizador (igual ao seu esquema A/B)
N_GLOBAL = 220
N_LOCAL  = 480
PATIENCE = 60
SHRINK   = 0.65

# -------------------- Leitura do cabeçalho do PARAMS_FILE --------------------
def read_params_header_and_name(params_file: str):
    with open(params_file, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    var_index = next(i for i, l in enumerate(lines) if l.strip().startswith('@VAR#'))
    header = lines[:var_index + 3]  # mantém o mesmo recorte usado por você
    disease_name = lines[var_index + 3].split()[1]
    return header, disease_name

# -------------------- DSSAT I/O --------------------
def write_params_to_file(params_all: Dict[str, float], header_lines: List[str], disease_name: str, sim_number: int = 1):
    p = params_all.copy()
    # coerções
    p['KVERHULST'] = int(round(p['KVERHULST']))
    p['NDS'] = int(p['NDS'])
    p['DAE_START'] = int(p['DAE_START'])
    # linha DSSAT
    new_id = f"D{sim_number:05d}"
    line = (
        f"{new_id:<7s} {disease_name:<17s}"
        f"{p['NDS']:>10d}{p['LESION_S']:>16.6f}"
        f"{p['KVERHULST']:>16d}{p['RVERHULST']:>16.3f}"
        f"{p['YMAX']:>16.3f}{p['COF_A']:>16.3f}{p['COF_B']:>16.3f}"
        f"{p['TMIN_G']:>16.3f}{p['TOT_G']:>16.3f}{p['TMAX_G']:>16.3f}"
        f"{p['TMIN_D']:>16.3f}{p['TOT_D']:>16.3f}{p['TMAX_D']:>16.3f}"
        f"{p['LDMin']:>16.3f}{p['LESIONAGEOPT']:>16.3f}{p['LESLIFEMAX']:>16.3f}"
        f"{p['DAE_START']:>16d}"
    )
    with open(PARAMS_FILE, 'w', encoding='utf-8') as f:
        f.writelines(header_lines)
        f.write(line + '\n')

def run_dssat_simulation(experiment_name: str) -> bool:
    try:
        subprocess.run(
            [DSSAT_EXE, "A", experiment_name],
            cwd=EXP_DIR, check=True, capture_output=True, text=True, encoding='latin1'
        )
        return True
    except (subprocess.CalledProcessError, FileNotFoundError):
        return False

# -------------------- Leitura do OUT e severidade --------------------
def read_disease_out_last_run(path_out: str) -> pd.DataFrame:
    with open(path_out, 'r', encoding='latin1', errors='ignore') as f:
        lines = f.readlines()
    header_idx = None
    target_keys = ["RUN", "YYDOY", "DAS", "LAI_HEALTH", "LA_DISEASE", "LAI_TOTAL"]
    for idx, ln in enumerate(lines):
        if all(k in ln for k in target_keys):
            header_idx = idx
            break
    if header_idx is None:
        raise ValueError("Cabeçalho não encontrado em DISEASE_DEVELOPMENT.OUT")

    colnames = [
        "RUN","YYDOY","DAS","LAI_HEALTH","LA_DISEASE","LA_INFECT","NEW_LOSS",
        "LWD","RH","FT","LAI_TOTAL","SUM7","NSPRAYS","FUNG_ACT"
    ]
    df = pd.read_fwf(path_out, skiprows=header_idx+1, names=colnames, engine="python", encoding='latin1')

    for c in ["RUN","YYDOY","DAS","LAI_HEALTH","LA_DISEASE","LA_INFECT","LAI_TOTAL","SUM7","NSPRAYS"]:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")

    df = df.dropna(subset=["RUN", "DAS"]).copy()
    if df.empty:
        raise ValueError("Sem linhas numéricas após o cabeçalho no OUT.")

    last_run = int(df["RUN"].max())
    df = df[df["RUN"] == last_run].copy()

    df["DAS"] = df["DAS"].astype(int)
    df = df.sort_values("DAS")

    keep = ["DAS","LAI_HEALTH","LAI_TOTAL","LA_DISEASE","LA_INFECT"]
    for k in keep:
        if k not in df.columns:
            df[k] = np.nan
    return df[keep].dropna(subset=["DAS"])

def calculate_severity(df_last_run: pd.DataFrame, last_obs_day: int = None) -> pd.DataFrame:
    df = df_last_run.copy()
    for c in ["DAS","LAI_TOTAL","LA_DISEASE"]:
        df[c] = pd.to_numeric(df[c], errors='coerce')
    df = df.dropna(subset=["DAS","LAI_TOTAL","LA_DISEASE"]).sort_values("DAS")
    if df.empty:
        return pd.DataFrame(columns=["DAS","Severity(%)"])

    df["DAS"] = df["DAS"].astype(int)
    if last_obs_day is not None and np.isfinite(last_obs_day):
        lai_peak = float(df.loc[df["DAS"] <= int(last_obs_day), "LAI_TOTAL"].max())
        if not np.isfinite(lai_peak) or lai_peak <= 0:
            lai_peak = float(df["LAI_TOTAL"].max())
    else:
        lai_peak = float(df["LAI_TOTAL"].max())

    lai_peak = max(lai_peak, 1e-9)
    sev = 100.0 * df["LA_DISEASE"].clip(lower=0.0) / lai_peak
    sev = np.clip(sev.values, 0.0, 100.0)
    sev = np.maximum.accumulate(sev)  # monotonicidade
    return pd.DataFrame({"DAS": df["DAS"].values, "Severity(%)": sev})

# -------------------- Métricas --------------------
def willmott_d(obs: np.ndarray, sim: np.ndarray) -> float:
    """Índice de concordância de Willmott (0..1, quanto mais perto de 1 melhor)."""
    y = np.asarray(obs, dtype=float); x = np.asarray(sim, dtype=float)
    if y.size < 2 or x.size != y.size: return np.nan
    ybar = float(np.mean(y))
    num  = np.sum((x - y)**2)
    den  = np.sum((np.abs(x - ybar) + np.abs(y - ybar))**2)
    return 1.0 - (num/den) if den > 0 else np.nan

def willmott_d1(obs: np.ndarray, sim: np.ndarray) -> float:
    """Índice de Willmott modificado (d1), mais robusto a outliers (0..1)."""
    y = np.asarray(obs, dtype=float); x = np.asarray(sim, dtype=float)
    if y.size < 2 or x.size != y.size: return np.nan
    ybar = float(np.mean(y))
    num  = np.sum(np.abs(x - y))
    den  = np.sum(np.abs(x - ybar) + np.abs(y - ybar))
    return 1.0 - (num/den) if den > 0 else np.nan

def eval_merge(sim_df: pd.DataFrame, obs_df: pd.DataFrame, curve_id):
    """
    Retorna: rmse, ef (Nash-Sutcliffe), merged_df, sim_df_filtrado, sse, n, d, d1
    """
    oc = obs_df[obs_df['Curva'] == curve_id].copy()
    if oc.empty:
        return np.inf, -np.inf, None, None, 0.0, 0, np.nan, np.nan
    oc['Av_DAS'] = pd.to_numeric(oc['Av_DAS'], errors='coerce')
    oc['Severidade (%)'] = pd.to_numeric(oc['Severidade (%)'], errors='coerce')
    oc.dropna(subset=['Av_DAS','Severidade (%)'], inplace=True)
    if len(oc) < 3:
        return np.inf, -np.inf, None, None, 0.0, 0, np.nan, np.nan

    ultimo = int(oc['Av_DAS'].max())
    sim_df_f = sim_df[sim_df['DAS'] <= ultimo].copy()

    obs_points = oc[['Av_DAS','Severidade (%)']].rename(columns={'Av_DAS':'DAS','Severidade (%)':'obs'})
    merged = pd.merge(sim_df_f, obs_points, on='DAS', how='inner')
    if len(merged) < 3:
        return np.inf, -np.inf, None, None, 0.0, 0, np.nan, np.nan

    resid = merged['obs'].values - merged['Severity(%)'].values
    sse = float(np.sum(resid**2))
    n   = len(merged)
    rmse = float(math.sqrt(sse / n))

    y = merged['obs'].values
    ybar = float(np.mean(y))
    sst  = float(np.sum((y - ybar)**2))
    ef = float(1.0 - sse / sst) if sst > 0 else -np.inf

    # Willmott
    d  = willmott_d(merged['obs'].values, merged['Severity(%)'].values)
    d1 = willmott_d1(merged['obs'].values, merged['Severity(%)'].values)

    return rmse, ef, merged, sim_df_f, sse, n, d, d1

def mse_decomposition(y: np.ndarray, x: np.ndarray):
    """
    Kobayashi & Salam (2000): MSE = (x̄ - ȳ)^2 + (sx - sy)^2 + 2*sx*sy*(1 - r)
    -> (bias², SDSD, LCS)
    """
    y = np.asarray(y, dtype=float)
    x = np.asarray(x, dtype=float)
    if len(y) < 2 or len(x) != len(y):
        return np.nan, np.nan, np.nan, np.nan
    xb, yb = np.mean(x), np.mean(y)
    sx, sy = np.std(x, ddof=1), np.std(y, ddof=1)
    r = 0.0 if np.isclose(sx,0.0) or np.isclose(sy,0.0) else float(np.corrcoef(x,y)[0,1])
    bias2 = (xb - yb)**2
    sdsd  = (sx - sy)**2
    lcs   = 2.0 * sx * sy * (1.0 - r)
    mse   = bias2 + sdsd + lcs
    return mse, bias2, sdsd, lcs

# -------------------- Amostradores: SÓ biológicos --------------------
def enforce_constraints_bio(p: Dict[str, float]) -> Dict[str, float]:
    # bounds
    for k, (a, b) in RANGES_BIO.items():
        if k in p: p[k] = float(np.clip(p[k], a, b))
    # ordenação T
    for (lo, mid, hi) in [('TMIN_G','TOT_G','TMAX_G'), ('TMIN_D','TOT_D','TMAX_D')]:
        v = sorted([p[lo], p[mid], p[hi]])
        p[lo], p[mid], p[hi] = v[0], v[1], v[2]
    # inteiro
    p['KVERHULST'] = int(round(p['KVERHULST']))
    return p

def sample_global_bio() -> Dict[str, float]:
    p = {}
    for k, (a, b) in RANGES_BIO.items():
        w = WEIGHTS.get(k, 0.5)
        u = a + (b - a) * random.random()
        x = w * u + (1.0 - w) * NOMINAL_BIO[k]
        p[k] = x
    return enforce_constraints_bio(p)

def sample_local_bio(best: Dict[str, float], base_scale: float = 0.30) -> Dict[str, float]:
    p = {}
    for k, (a, b) in RANGES_BIO.items():
        width = (b - a)
        sigma = base_scale * width * WEIGHTS.get(k, 0.5)
        mutate_prob = 0.20 + 0.80 * WEIGHTS.get(k, 0.5)
        p[k] = np.random.normal(loc=best[k], scale=max(sigma, 1e-9)) if random.random() < mutate_prob else best[k]
    return enforce_constraints_bio(p)

# -------------------- Objetivo de TREINO (P-1), usando NDS/DAE da planilha --------------------
def objective_train_bio(params_bio: Dict[str, float],
                        header: List[str],
                        disease_name: str,
                        train_df: pd.DataFrame,
                        observed_df: pd.DataFrame,
                        run_id_start: int = 1):
    """
    Roda todos os experimentos de treino com:
       params_all = params_bio (compartilhado) + NDS/DAE_START (de cada curva)
    Agrega SSE e n (pontos de comparação).
    """
    SSE_total, N_total, n_ok = 0.0, 0, 0
    run_id = run_id_start

    for _, ex in train_df.iterrows():
        exp_name = str(ex['Experiment'])
        if not exp_name.endswith(".SBX"):
            exp_name = f"{exp_name}.SBX"

        params_all = params_bio.copy()
        params_all['NDS'] = int(ex['NDS'])
        params_all['DAE_START'] = int(ex['DAE_START'])

        write_params_to_file(params_all, header, disease_name, sim_number=run_id)
        run_id += 1
        if not run_dssat_simulation(exp_name):
            continue

        if os.path.exists(DSSAT_OUTPUT_FILE):
            try:
                df_out = read_disease_out_last_run(DSSAT_OUTPUT_FILE)
                curve_id = ex['Id_curva']
                oc = observed_df[observed_df['Curva'] == curve_id]
                last_obs_day = int(pd.to_numeric(oc['Av_DAS'], errors='coerce').max()) if not oc.empty else None
                sim_sev = calculate_severity(df_out, last_obs_day=last_obs_day)
                # d/d1 são calculados mas não usados para a função objetivo (mantida a lógica)
                rmse, _, merged_df, _, sse, n, _, _ = eval_merge(sim_sev, observed_df, curve_id)
                if merged_df is not None and n >= 3 and np.isfinite(rmse):
                    SSE_total += sse
                    N_total   += n
                    n_ok      += 1
            except Exception:
                continue

    if N_total == 0 or n_ok == 0:
        return np.inf, 0.0, 0, 0
    rmse_train = math.sqrt(SSE_total / N_total)
    return rmse_train, SSE_total, N_total, n_ok

def train_bio_on_fold(train_df: pd.DataFrame,
                      observed_df: pd.DataFrame,
                      header: List[str],
                      disease_name: str,
                      verbose: bool = True) -> Dict[str, float]:
    """
    Otimiza somente BIOLÓGICOS com P-1 curvas (NDS/DAE vêm do control).
    """
    best_params, best_rmse = None, np.inf
    run_id = 10000

    # Fase A
    for i in range(1, N_GLOBAL + 1):
        cand = sample_global_bio()
        rmse_tr, _, n_tot, n_ok = objective_train_bio(cand, header, disease_name, train_df, observed_df, run_id_start=run_id)
        run_id += len(train_df)
        if np.isfinite(rmse_tr) and rmse_tr < best_rmse:
            best_rmse, best_params = rmse_tr, cand.copy()
            if verbose:
                print(f"[Fold-Train A {i:03d}/{N_GLOBAL}] RMSE_treino={best_rmse:.4f} (pts: {n_tot}, exps ok: {n_ok}/{len(train_df)})")

    if best_params is None:
        raise RuntimeError("Nenhuma solução válida na Fase A (treino).")

    # Fase B
    base_scale, no_improve = 0.30, 0
    for j in range(1, N_LOCAL + 1):
        cand = sample_local_bio(best_params, base_scale=base_scale)
        rmse_tr, _, n_tot, n_ok = objective_train_bio(cand, header, disease_name, train_df, observed_df, run_id_start=run_id)
        run_id += len(train_df)
        if np.isfinite(rmse_tr) and rmse_tr < best_rmse:
            best_rmse, best_params, no_improve = rmse_tr, cand.copy(), 0
            if verbose:
                print(f"[Fold-Train B {j:03d}/{N_LOCAL}] >>> RMSE_treino={best_rmse:.4f} (pts: {n_tot}, exps ok: {n_ok}/{len(train_df)})")
        else:
            no_improve += 1
            if no_improve % PATIENCE == 0:
                base_scale = max(base_scale * SHRINK, 0.05)
                if verbose:
                    print(f"(sem melhora por {PATIENCE}) shrink -> base_scale={base_scale:.3f}")
    return best_params

def evaluate_left_out(test_row: pd.Series,
                      best_bio_params: Dict[str, float],
                      header: List[str],
                      disease_name: str,
                      observed_df: pd.DataFrame,
                      sim_id: int):
    """
    Teste na curva deixada de fora com biológicos treinados + NDS/DAE da própria curva.
    Retorna dict com métricas, incluindo decomposição MSE e Willmott d/d1.
    """
    exp_name = str(test_row['Experiment'])
    if not exp_name.endswith(".SBX"):
        exp_name = f"{exp_name}.SBX"

    params_all = best_bio_params.copy()
    params_all['NDS'] = int(test_row['NDS'])
    params_all['DAE_START'] = int(test_row['DAE_START'])

    write_params_to_file(params_all, header, disease_name, sim_number=sim_id)
    if not run_dssat_simulation(exp_name) or not os.path.exists(DSSAT_OUTPUT_FILE):
        return {"Id_curva": test_row['Id_curva'], "rmse": np.nan, "ef": np.nan, "d": np.nan, "d1": np.nan, "n": 0,
                "bias2": np.nan, "SDSD": np.nan, "LCS": np.nan, "mse": np.nan, "sse": np.nan}

    try:
        df_out = read_disease_out_last_run(DSSAT_OUTPUT_FILE)
        curve_id = test_row['Id_curva']
        oc = observed_df[observed_df['Curva'] == curve_id]
        last_obs_day = int(pd.to_numeric(oc['Av_DAS'], errors='coerce').max()) if not oc.empty else None
        sim_sev = calculate_severity(df_out, last_obs_day=last_obs_day)
        rmse, ef, merged, _, sse, n, d, d1 = eval_merge(sim_sev, observed_df, curve_id)
        if merged is None or n < 3:
            return {"Id_curva": curve_id, "rmse": np.nan, "ef": np.nan, "d": np.nan, "d1": np.nan, "n": 0,
                    "bias2": np.nan, "SDSD": np.nan, "LCS": np.nan, "mse": np.nan, "sse": np.nan}
        y = merged['obs'].values
        x = merged['Severity(%)'].values
        mse, bias2, sdsd, lcs = mse_decomposition(y, x)
        return {"Id_curva": curve_id, "rmse": rmse, "ef": ef, "d": d, "d1": d1, "n": n,
                "bias2": bias2, "SDSD": sdsd, "LCS": lcs, "mse": mse, "sse": sse}
    except Exception:
        return {"Id_curva": test_row['Id_curva'], "rmse": np.nan, "ef": np.nan, "d": np.nan, "d1": np.nan, "n": 0,
                "bias2": np.nan, "SDSD": np.nan, "LCS": np.nan, "mse": np.nan, "sse": np.nan}

# -------------------- MAIN --------------------
def main():
    # Carrega entradas
    try:
        control_df  = pd.read_excel(CONTROL_SHEET_PATH)
        observed_df = pd.read_excel(OBSERVED_DATA_PATH, sheet_name=OBSERVED_SHEET_NAME)
        header, disease_name = read_params_header_and_name(PARAMS_FILE)
    except Exception as e:
        print(f"[ERRO] Carregando entradas: {e}")
        return

    # Checa colunas obrigatórias (agora padronizadas)
    needed = ['Id_curva','Experiment','NDS','DAE_START']
    for col in needed:
        if col not in control_df.columns:
            raise KeyError(f"Coluna obrigatória ausente em control: {col}")

    curves = list(control_df['Id_curva'].unique())
    print(f"\n==== LOOCV região '{REGION_NAME}' | {len(curves)} curvas ====\n")

    results = []
    all_obs_concat, all_sim_concat = [], []

    for i, cv_id in enumerate(curves, start=1):
        print(f"\n--- Fold {i}/{len(curves)} | left-out: Curva {cv_id} ---")
        left_mask = control_df['Id_curva'] == cv_id
        train_df = control_df[~left_mask].copy()
        test_row = control_df[left_mask].iloc[0]

        # Treino (P-1) -> biológicos
        best_bio = train_bio_on_fold(train_df, observed_df, header, disease_name, verbose=True)

        # Teste (curva deixada)
        metrics = evaluate_left_out(test_row, best_bio, header, disease_name, observed_df, sim_id=900000+i)
        results.append(metrics)

        # Para decomposição global, rodamos de novo (ou reaproveitamos) e concatenamos pontos alinhados
        exp_name = str(test_row['Experiment'])
        if not exp_name.endswith(".SBX"):
            exp_name = f"{exp_name}.SBX"
        params_all = best_bio.copy()
        params_all['NDS'] = int(test_row['NDS'])
        params_all['DAE_START'] = int(test_row['DAE_START'])
        write_params_to_file(params_all, header, disease_name, sim_number=910000+i)
        if run_dssat_simulation(exp_name) and os.path.exists(DSSAT_OUTPUT_FILE):
            try:
                df_out = read_disease_out_last_run(DSSAT_OUTPUT_FILE)
                oc = observed_df[observed_df['Curva'] == cv_id]
                last_obs_day = int(pd.to_numeric(oc['Av_DAS'], errors='coerce').max()) if not oc.empty else None
                sim_sev = calculate_severity(df_out, last_obs_day=last_obs_day)
                rmse, ef, merged, _, sse, n, _, _ = eval_merge(sim_sev, observed_df, cv_id)
                if merged is not None and len(merged) >= 3:
                    all_obs_concat.append(merged['obs'].values)
                    all_sim_concat.append(merged['Severity(%)'].values)
            except Exception:
                pass

    # Resultados por curva
    df_cv = pd.DataFrame(results)
    df_cv_path = OUTPUT_DIR / f"cv_loocv_results_{REGION_NAME}.csv"
    df_cv.to_csv(df_cv_path, index=False, encoding="utf-8-sig")
    print(f"\n[OK] Resultados por curva salvos em: {df_cv_path}")

    # Agregados
    sse_sum = float(np.nansum(df_cv.get('sse', pd.Series(dtype=float)).values)) if 'sse' in df_cv else np.nan
    n_sum   = int(np.nansum(df_cv.get('n',   pd.Series(dtype=float)).values))   if 'n' in df_cv else 0
    RMSEp   = float(math.sqrt(sse_sum / n_sum)) if n_sum > 0 and np.isfinite(sse_sum) else np.nan
    EF_mean = float(np.nanmean(df_cv['ef'].values)) if 'ef' in df_cv else np.nan

    if len(all_obs_concat) > 0:
        Y_all = np.concatenate(all_obs_concat)
        X_all = np.concatenate(all_sim_concat)
        MSE_g, BIAS2_g, SDSD_g, LCS_g = mse_decomposition(Y_all, X_all)
        D_g   = willmott_d(Y_all, X_all)
        D1_g  = willmott_d1(Y_all, X_all)
    else:
        MSE_g = BIAS2_g = SDSD_g = LCS_g = np.nan
        D_g = D1_g = np.nan

    summary = {
        "REGION": REGION_NAME,
        "RMSEp": RMSEp,
        "EF_mean": EF_mean,
        "MSE_global": MSE_g,
        "bias2_global": BIAS2_g,
        "SDSD_global": SDSD_g,
        "LCS_global": LCS_g,
        "D_global": D_g,
        "D1_global": D1_g,
        "n_total": n_sum,
        "num_curvas": len(curves)
    }
    df_sum = pd.DataFrame([summary])
    df_sum_path = OUTPUT_DIR / f"cv_loocv_summary_{REGION_NAME}.csv"
    df_sum.to_csv(df_sum_path, index=False, encoding="utf-8-sig")
    print(f"[OK] Sumário global salvo em: {df_sum_path}")

    print("\n==== SUMÁRIO LOOCV ====")
    print(f"Region: {REGION_NAME}")
    print(f"Curvas: {len(curves)} | n_total: {n_sum}")
    print(f"RMSEp: {RMSEp:.3f}" if np.isfinite(RMSEp) else "RMSEp: NaN")
    print(f"EF_mean: {EF_mean:.3f}" if np.isfinite(EF_mean) else "EF_mean: NaN")
    print(f"Decomposição MSE global -> bias²: {BIAS2_g:.3f}, SDSD: {SDSD_g:.3f}, LCS: {LCS_g:.3f}")
    print(f"Willmott -> d: {D_g:.3f}, d1: {D1_g:.3f}")

if __name__ == "__main__":
    main()



==== LOOCV região 'sul' | 6 curvas ====


--- Fold 1/6 | left-out: Curva 1 ---
[Fold-Train A 001/220] RMSE_treino=17.3005 (pts: 29, exps ok: 5/5)
[Fold-Train A 002/220] RMSE_treino=14.9266 (pts: 29, exps ok: 5/5)
[Fold-Train A 008/220] RMSE_treino=13.1652 (pts: 29, exps ok: 5/5)
[Fold-Train A 011/220] RMSE_treino=11.6178 (pts: 29, exps ok: 5/5)
[Fold-Train A 026/220] RMSE_treino=10.2988 (pts: 29, exps ok: 5/5)
[Fold-Train A 030/220] RMSE_treino=10.2211 (pts: 29, exps ok: 5/5)
[Fold-Train A 105/220] RMSE_treino=9.4991 (pts: 29, exps ok: 5/5)
[Fold-Train A 130/220] RMSE_treino=9.0892 (pts: 29, exps ok: 5/5)
[Fold-Train B 001/480] >>> RMSE_treino=8.2867 (pts: 29, exps ok: 5/5)
[Fold-Train B 003/480] >>> RMSE_treino=8.1722 (pts: 29, exps ok: 5/5)
(sem melhora por 60) shrink -> base_scale=0.195
[Fold-Train B 076/480] >>> RMSE_treino=8.1247 (pts: 29, exps ok: 5/5)
[Fold-Train B 130/480] >>> RMSE_treino=8.0786 (pts: 29, exps ok: 5/5)
[Fold-Train B 135/480] >>> RMSE_treino=7.6999 (pts: 29, ex